In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import yagmail
from requests_futures.sessions import FuturesSession
from concurrent.futures import ProcessPoolExecutor
import time

In [2]:
##Some urls may have a .json on the end, add where needed

In [3]:
# Add either a website/base url or ipfs url
start = datetime.now()

#Add token supply and the first id of the collection on opensea
max_supply = 4000
token_id_start = 1

if token_id_start == 0:
    max_supply = max_supply-1
elif token_id_start == 1:
    max_supply = max_supply
    
contract_address = '0x3B8420eB6EF77Bc25025b32D8205cde2683F979E'
base_url = 'https://moonbears.mypinata.cloud/ipfs/QmXMJ8Eysnx9Qwk1Wozv7UbGKvwr7r7sP2oUg5gwEd3z9R/'

ipfs_token = 'Qmf4aGqMouRdV1u8tvVWN37g6gnw7LP6tcJdsuE6QUXBcu/'
ipfs_url = f'https://ipfs.io/ipfs/{ipfs_token}'

In [4]:
#Testing all metadata has been released

def test_first(url, token_id):
    url =  f'{url}{token_id}'
    print(url)
    r = requests.get(url)
    json_data = json.loads(r.text)

    return (f'The first nft metadata is {json_data}')

def test_last(url, max_supply):
    url =  f'{url}{max_supply}'
    r = requests.get(url)
    print(url)
    json_data = json.loads(r.text)
    
    return (f'The last nft metadata is: {json_data}')

def test_urls(url):
    print(test_first(url, token_id_start))
    print(test_last(url, max_supply))
    
test_urls(base_url)

https://moonbears.mypinata.cloud/ipfs/QmXMJ8Eysnx9Qwk1Wozv7UbGKvwr7r7sP2oUg5gwEd3z9R/1
The first nft metadata is {'name': 'Moonbear #1', 'image': 'ipfs://QmSrKVqjj2EwKMpkKGxDBQx6nXAbcAKDaP3sJW32p47Pad/1.png', 'attributes': [{'trait_type': 'Background', 'value': 'Blue'}, {'trait_type': 'Body', 'value': 'Brown'}, {'trait_type': 'Mouth', 'value': 'Cute'}, {'trait_type': 'Eyes', 'value': 'Golden'}, {'trait_type': 'Clothing', 'value': 'Green T Shirt'}, {'trait_type': 'Eyewear', 'value': 'None'}, {'trait_type': 'Headwear', 'value': 'Bandana'}]}
https://moonbears.mypinata.cloud/ipfs/QmXMJ8Eysnx9Qwk1Wozv7UbGKvwr7r7sP2oUg5gwEd3z9R/4000
The last nft metadata is: {'name': 'Moonbear #4000', 'image': 'ipfs://QmSrKVqjj2EwKMpkKGxDBQx6nXAbcAKDaP3sJW32p47Pad/4000.png', 'attributes': [{'trait_type': 'Background', 'value': 'Peach'}, {'trait_type': 'Body', 'value': 'Jungle'}, {'trait_type': 'Mouth', 'value': 'Surprise'}, {'trait_type': 'Eyes', 'value': 'Diagonal'}, {'trait_type': 'Clothing', 'value': 'Gre

In [5]:
#Scraping all metadata

data = []

def scrape_metadata(used_url, token_id_start):
    t1 = datetime.now()
    list_url = []

    for token in range(token_id_start, max_supply):
        url = f'{used_url}{token}'
        list_url.append(url)
        
    print('Urls being processed...')
    # Multiple sessions at once
    
    number_sessions = []
    
    with FuturesSession(max_workers = 32) as session:
        futures = [session.get(url) for url in list_url]
        for future in futures:
            # print('Session being pushed')
            number_sessions.append('s')
            percent = (len(number_sessions)/len(list_url))*100
            # print(f'{percent}% loaded..')
            response = future.result()
            data.append(json.loads(response.text))
            # response.close()
            
            interval = 100/max_supply
            print(percent)
    
            
    t2 = datetime.now()
    print(t2-t1)  

scrape_metadata(base_url, token_id_start)

Urls being processed...
0.025006251562890724
0.05001250312578145
0.07501875468867217
0.1000250062515629
0.12503125781445362
0.15003750937734434
0.17504376094023505
0.2000500125031258
0.22505626406601648
0.25006251562890724
0.27506876719179796
0.30007501875468867
0.3250812703175794
0.3500875218804701
0.3750937734433608
0.4001000250062516
0.42510627656914224
0.45011252813203295
0.4751187796949237
0.5001250312578145
0.5251312828207052
0.5501375343835959
0.5751437859464866
0.6001500375093773
0.6251562890722681
0.6501625406351588
0.6751687921980495
0.7001750437609402
0.7251812953238309
0.7501875468867216
0.7751937984496124
0.8002000500125032
0.8252063015753939
0.8502125531382845
0.8752188047011753
0.9002250562640659
0.9252313078269567
0.9502375593898474
0.9752438109527383
1.000250062515629
1.0252563140785198
1.0502625656414104
1.0752688172043012
1.1002750687671918
1.1252813203300824
1.1502875718929733
1.1752938234558639
1.2003000750187547
1.2253063265816453
1.2503125781445361
1.275318829707

## Sorting data and mapping traits

In [6]:
# Other idea map the the traits and value to each other then use count
mapped_traits = []
nft_id = []

#Maybe remove the '-token_id_start'
def map_trait_and_value(token_id_start):
    for number in range(0, max_supply-token_id_start):
        attributes = data[number]['attributes']
        number_of_attributes = len(attributes)
        
        
        for att in range(0, number_of_attributes):
            trait = attributes[att]['trait_type']
            value = attributes[att]['value']
        
            nft_id.append(number+token_id_start)
            
            mapped = (f'{trait}-{value}')
            mapped_traits.append(mapped)
    
map_trait_and_value(token_id_start)

In [10]:
#Making a mapped df
mapped_df = pd.DataFrame(mapped_traits)

unique_counts = pd.DataFrame(mapped_df.value_counts())
unique_counts.rename(columns={0:'count'}, inplace = True)
unique_counts.reset_index(level=0, inplace = True)
unique_counts.rename(columns={0:'attribute_value'}, inplace = True)
unique_counts = unique_counts[unique_counts['count'] <=40]

#Making a df with all nfts ids + traits
all_nfts = pd.DataFrame(nft_id, mapped_traits)
all_nfts.rename(columns={0:'id'}, inplace = True)
all_nfts.reset_index(level=0, inplace = True)
all_nfts.rename(columns={'index':'attribute_value'}, inplace = True)

In [13]:
# Intesection
open_sea_urls_rarest_top_50 = []
top_x_rarest = 50

def find_rarest(top_x_rarest, contract_address):
    best_attributes_in_all_nfts = all_nfts[all_nfts['attribute_value'].isin(unique_counts['attribute_value'])]

    df = pd.DataFrame(best_attributes_in_all_nfts.id.value_counts())
    rarest = df.sort_values(by='id', ascending = False).head(top_x_rarest)
    
   
    for token_id in rarest.index:
        open_sea_url = f'https://opensea.io/assets/{contract_address}/{token_id}'
        open_sea_urls_rarest_top_50.append(open_sea_url)
        
    return rarest

find_rarest(top_x_rarest, contract_address)
top = len(open_sea_urls_rarest_top_50)


36

In [14]:
## Emailing the results

#Timing the run
end = datetime.now()
time_to_run = end-start

#For this I made a new gmail and sent myself a email with the results in.
user = yagmail.SMTP(user='YOUR_EMAIL_ADDRESS_HERE', password='EMAIL_ADDRESS_PASSWORD')

#Features
time = datetime.now().strftime('%Y-%m-%d / %H:%M:%S')
etherscan = 'https://etherscan.io/address/'
intro = (f'Metadata scrape completed on {time} taking {time_to_run}.')

#Body of email
body_1 = f'\nContract address - {etherscan}{contract_address}'
body_2 = f'\nMax supply = {max_supply}\n Token ID starts at {token_id_start}'
body_3 = f'\nTop {top} NFTs in this collection:'

user.send(to=('RECIPIENT_EMAIL')
          ,subject =f'Metadata - {time}',
          contents=(intro, body_1, body_2, body_3, open_sea_urls_rarest_top_50))

{}